# Feature Analysis

A notebook to find example pairs of ranked specs where both specs have the same set of features. This is not good as w cannot learn a difference. 

In [1]:
from draco.learn.data_util import load_neg_pos_data, get_pos_neg_data
import json
from vega3 import VegaLite
from pprint import pprint

In [2]:
raw_data = load_neg_pos_data()

In [3]:
raw_data[0]

PosNeg(data=<draco.spec.Data object at 0x11745cf98>, task=None, source='manual', negative={'mark': 'point', 'encoding': {'x': {'field': 'q1', 'type': 'quantitative'}, 'y': {'field': 'q1', 'type': 'quantitative'}}}, positive={'mark': 'point', 'encoding': {'x': {'field': 'q1', 'type': 'quantitative'}, 'y': {'field': 'q2', 'type': 'quantitative'}}})

In [4]:
data = get_pos_neg_data()

In [5]:
residual = data.positive - data.negative

In [6]:
sums = residual.abs().sum(axis=1)

In [7]:
bad = data[sums == 0]
print(len(bad))

0


In [8]:
bad.index

Int64Index([], dtype='int64')

In [ ]:
# bad.positive.iloc[0]

In [ ]:
# bad.negative.iloc[0]

In [ ]:
e = raw_data[bad.index[0]]
print(e.data.to_asp())
pprint(e.negative)
pprint(e.positive)